Neste trabalho iremos fazer a busca dos melhores hiperparametros de uma SVM para Regressão num banco de dados em particular. Para o desenvolvimento utilizaremos a biblioteca sklearn.svm que apresenta a função  SVR() que implementa o regressor SVM e tem vários hiperparametros.
Vamos usar o kernel “rbf”, havendo 3 hiperparametros que consideramos como os mais importantes: C, gamma, e epsilon.

Vamos fazer a busca no range:
    - C entre 2^{-5} e 2^{15}  (uniforme nos expoentes);
    - gamma entre 2^{15} e 2^{-3} (uniforme nos expoentes);
    - episolon entre 0.05 e 10  (uniforme neste intervalo);
    
Utilizamos como dados de treino e testes os arquivos Xtreino5.npy e Xteste5.npy. Para as sáidas dos dados correspondenetres usamos os arquivos ytreino5.npy e yteste5.npy. Todos os arquivos foram disponibilizados na página no trabalho https://www.ic.unicamp.br/~wainer/cursos/1s2020/431/ex4.html.

### Importação das Bibliotecas

Como primeiro passo, importamos as bibliotecas necessárias para a implementação desse trabalho: numpy e sklearn.svm.

In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
import pyspark
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

from pyswarm import pso
from simanneal import Annealer
import optuna
from scipy.stats import loguniform
import random

### Leitura e Exibição dos arquivos

Após importamos as bibliotecas necessárias, realizamos a leitura dos arquivos .npy disponibilizados para uso neste trabalho. Para isso fizemos uso da função load() da biblioteca numpy. Armazenamos cada um dos arquivos de treino e de teste em variáveis locais.

In [2]:
x_treino = np.load('data/Xtreino5.npy')
x_teste = np.load('data/Xteste5.npy')
y_treino = np.load('data/ytreino5.npy')
y_teste = np.load('data/yteste5.npy')

### Medida de erro

Como primeira tarefa, para cada conjunto de hiperparametros, treinamos o SVM no conjunto de treino (Xtreino e ytreino), e medimos o erro absoluto médio (MAE) no conjunto de teste (Xteste e yteste).

## Randomized Search

In [3]:
n_combinations = 125

# Hyperparameters Search Space
C_range = np.random.uniform(-5, 15, n_combinations).astype(float)
C_range = 2**C_range

# C_range = loguniform(2**-5, 2**15).rvs(size=n_combinations)

gamma_range = np.random.uniform(-15, 3, n_combinations).astype(float)
gamma_range = 2**gamma_range

# gamma_range = loguniform(2**-15, 2** 3).rvs(size=n_combinations)

epsilon_range = np.random.uniform(0.05, 1.0, n_combinations).astype(float)

 
hyperparameters = {'gamma': list(gamma_range), 
                    'C': list(C_range),
                  'epsilon': list(epsilon_range)}
 
# print (hyperparameters)

In [4]:
# Run randomized search
randomCV = RandomizedSearchCV(SVR(kernel='rbf'), param_distributions=hyperparameters, n_iter=20)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']

print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))


The best performing C value is: 203.58
The best performing gamma value is: 0.00004
The best performing epsilon value is:  0.10


In [5]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  3.5179791474928157


### Grid seach

O próximo algoritimo de otimização proposto foi o Grid Search. Seguindo a especificação de uma busca em uma grid de 5x5x5, amostras do range de busca foram tomadas:

In [26]:
# Hyperparameters Search Space
n_combinations = 5

# Taken 5 samples randomly
grid_c = random.sample(list(C_range), k=n_combinations)
grid_gamma = random.sample(list(gamma_range), k=n_combinations)
grid_episolon = random.sample(list(epsilon_range), k=n_combinations)

# The best performing gamma value is: 0.00021
# The best performing C value is: 1425.76
# The best performing epsilon value is:  0.91

# Generate a uniform distribution with 5 elements
# grid_c = np.random.uniform(-5, 15, n_combinations).astype(float)
# grid_c = 2**grid_c

# grid_gamma = np.random.uniform(-15, 3, n_combinations).astype(float)
# grid_gamma = 2**grid_gamma

# grid_episolon = np.random.uniform(0.05, 1.0, n_combinations).astype(float)

# The best performing gamma value is: 0.01466
# The best performing C value is: 593.42
# The best performing epsilon value is:  0.16

hyperparameters_grid = {'gamma': list(grid_gamma), 
                        'C': list(grid_c),
                        'epsilon': list(grid_episolon)}

In [27]:
# Run Grid Search
randomCV = GridSearchCV(SVR(kernel='rbf'), param_grid=hyperparameters_grid, cv = 5)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

The best performing gamma value is: 0.00021
The best performing C value is: 1425.76
The best performing epsilon value is:  0.91


In [8]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  3.125782711726864


### Otimização bayesiana

Outro algoritimo proposto foi a otimização bayesiana. Para a sua implementação utilziamos a biblioteca hyperopt, que dispnibiliza O regressor (TPE) para modelar a distribuição de probabilidades que é muito mais rápido que a implementação padrão utilizando “processos gaussianos”.

Esta implementação foi feita em passos:

### Definir a Função de mínimo

Como queremos pesquisar por Support Vector Machines (SVM), definimos um parâmetro params ['type'] como o nome do modelo, e uma função para executar o treinamento e retornar a precisão da validação cruzada. 
Como estamos tentando maximizar a precisão da validação cruzada, devemos negar esse valor para o hyperopt, pois o hyperopt sabe apenas como minimizar uma função.

In [9]:
def get_acc_status(clf,X_,y):
    acc = cross_val_score(clf, X_, y, cv=5).mean()
    return {'loss': -acc, 'status': STATUS_OK}

In [10]:
def objective(params):
    classifier_type = params['type']
    del params['type']
    if classifier_type == 'svm':
        clf = SVR(**params)
    else:
        return 0
    accuracy = cross_val_score(clf, x_treino, y_treino).mean()
    
    return {'loss': -accuracy, 'status': STATUS_OK}

### Definir espaço de pesquisa sobre os hiperparâmetros

In [11]:
search_space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        'C': hp.uniform('C', (2**-5), (2**15)),
        'gamma': hp.uniform('gamma', (2**-15), (2**3)),
        'epsilon': hp.uniform('epsilon', 0.05, 1.0),
        'kernel': hp.choice('kernel', ['rbf'])
    },
])

### Selecionar um algoritimo de busca

As duas opções principais de algoritmos de busca são:

    - hyperopt.tpe.suggest: Estimadores da Árvore de Parzen, uma abordagem bayesiana que seleciona iterativa e adaptativamente novas configurações de hiperparâmetro para explorar com base em resultados anteriores;
    - hyperopt.rand.suggest: Pesquisa aleatória, uma abordagem não adaptativa que mostra o espaço de pesquisa.
    
Conforme pedido, utilziamos o algoritimo TPE.

### Executar o algoritmo de ajuste com hyperopt fmin ()

Definimos max_evals como o número máximo de pontos no espaço do hiperparâmetro para testar, ou seja, o número máximo de modelos para ajustar e avaliar.

O SparkTrials usa 2 argumentos:
    - parallelism: Número de modelos para ajustar e avaliar simultaneamente.
    - timeout: tempo máximo (em segundos) que fmin pode demorar. Este argumento é opcional.

O rastreamento automatizado do MLflow está ativado por padrão. Ligue para mlflow.start_run () antes de chamar fmin (), como mostra o exemplo abaixo.

In [12]:
hypopt_trials = Trials()
 
best_params = fmin(objective, search_space, algo=tpe.suggest, 
max_evals=125, trials= hypopt_trials)
 
best_gamma  = best_params['gamma']
best_C      = best_params['C']
best_epsilon= best_params['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

100%|██████████| 125/125 [00:10<00:00, 12.12trial/s, best loss: -0.5333664926129773]
The best performing gamma value is: 0.00405
The best performing C value is: 1439.71
The best performing epsilon value is:  0.89


In [13]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  4.1689907232274175


## PSO

In [14]:
# PARAMETERS
C_MIN = 2**(-5)
C_MAX = 2**15

GAMMA_MIN = 2**(-15)
GAMMA_MAX = 2**3

EPSILON_MIN = 0.05
EPSILON_MAX = 1.0

lb = np.array([C_MIN, GAMMA_MIN, EPSILON_MIN])
ub = np.array([C_MAX, GAMMA_MAX, EPSILON_MAX])

# FUNCTION
def svr_fun(X):
    c = X[0]
    g = X[1]
    eps = X[2]
    
    svr  = SVR(kernel='rbf', C=c, gamma=g, epsilon=eps)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae

print("PSO...")
x_opt, y_opt = pso(svr_fun, lb, ub, swarmsize=11, maxiter=11)

print(" C optimal: "+ str(x_opt[0])+
     "\n Gamma Optimal: "+ str(x_opt[1])+
     "\n Epsilon Optimal: "+ str(x_opt[2]))
print("MAE: ", str(y_opt))

PSO...
Stopping search: maximum iterations reached --> 11
 C optimal: 11310.702824752187
 Gamma Optimal: 3.0517578125e-05
 Epsilon Optimal: 0.31212512970162964
MAE:  2.294919070973627


## Simulated Annealing

Para a implementação dos Métodos de Simmulate Annealing e CMA-ES foi utilizado a biblioteca optuna.

Para o método de Simulated Annealing a biblioteca fornece uma implementação via classe através dos chamados "samplers" para determinar os valores dos parâmetros a serem avaliados durante o teste. Para o Simmulated Annealing o sampler é implementado via classe explicitamente (vale ressaltar que estamos utilizando a implementação proposta pela própria documentação).

In [15]:
class SimulatedAnnealingSampler(optuna.samplers.BaseSampler):
    def __init__(self, temperature=100):
        self._rng = np.random.RandomState()
        self._temperature = temperature  # Current temperature.
        self._current_trial = None  # Current state.

    def sample_relative(self, study, trial, search_space):
        if search_space == {}:
            return {}

        #
        # An implementation of SA algorithm.
        #

        # Calculate transition probability.
        prev_trial = study.trials[-2]
        if self._current_trial is None or prev_trial.value <= self._current_trial.value:
            probability = 1.0
        else:
            probability = np.exp((self._current_trial.value - prev_trial.value) / self._temperature)
        self._temperature *= 0.9  # Decrease temperature.

        # Transit the current state if the previous result is accepted.
        if self._rng.uniform(0, 1) < probability:
            self._current_trial = prev_trial

        # Sample parameters from the neighborhood of the current point.
        #
        # The sampled parameters will be used during the next execution of
        # the objective function passed to the study.
        params = {}
        for param_name, param_distribution in search_space.items():
            if not isinstance(param_distribution, optuna.distributions.UniformDistribution):
                raise NotImplementedError('Only suggest_uniform() is supported')

            current_value = self._current_trial.params[param_name]
            width = (param_distribution.high - param_distribution.low) * 0.1
            neighbor_low = max(current_value - width, param_distribution.low)
            neighbor_high = min(current_value + width, param_distribution.high)
            params[param_name] = self._rng.uniform(neighbor_low, neighbor_high)

        return params

    #
    # The rest is boilerplate code and unrelated to SA algorithm.
    #
    def infer_relative_search_space(self, study, trial):
        return optuna.samplers.intersection_search_space(study)

    def sample_independent(self, study, trial, param_name, param_distribution):
        independent_sampler = optuna.samplers.RandomSampler()
        return independent_sampler.sample_independent(study, trial, param_name, param_distribution)





Por fim o é necessário criar uma função objetivo a qual deseja-se otimizar, em nosso caso procuramos minimizar o Erro Médio Absoluto (MAE) com uma validação no conjunto de testes da aplicação dos hyperparâmetros encontrados para o problema do SVM Regressor.

In [16]:
def objective(trial):
    c = trial.suggest_uniform('c', 2**(-5),  2**15)
    gamma = trial.suggest_uniform('gamma', 2**(-15),  2**3)    
    epsilon = trial.suggest_uniform('epsilon', 0.05,  1.0)
    
    svr  = SVR(kernel='rbf', C=c, gamma=gamma, epsilon=epsilon)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae


sampler = SimulatedAnnealingSampler()
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=125)

[I 2020-05-06 08:35:33,214] Finished trial#0 with value: 5.779741223206209 with parameters: {'gamma': 7.922091409550109, 'c': 23164.392036329296, 'epsilon': 0.9063545432872883}. Best is trial#0 with value: 5.779741223206209.
[I 2020-05-06 08:35:33,313] Finished trial#1 with value: 5.778987068264555 with parameters: {'gamma': 7.893022210570613, 'c': 24308.26755972299, 'epsilon': 0.8860306004199994}. Best is trial#1 with value: 5.778987068264555.
[I 2020-05-06 08:35:33,399] Finished trial#2 with value: 5.777997028769795 with parameters: {'gamma': 7.204369796172042, 'c': 22932.282935682775, 'epsilon': 0.8631791868499427}. Best is trial#2 with value: 5.777997028769795.
[I 2020-05-06 08:35:33,487] Finished trial#3 with value: 5.777047263160402 with parameters: {'gamma': 7.724072788773284, 'c': 24186.57349188241, 'epsilon': 0.8334495812680451}. Best is trial#3 with value: 5.777047263160402.
[I 2020-05-06 08:35:33,577] Finished trial#4 with value: 5.773753283195609 with parameters: {'gamma': 

[I 2020-05-06 08:35:36,086] Finished trial#35 with value: 5.778796658729584 with parameters: {'gamma': 6.590026640225364, 'c': 20583.200964201533, 'epsilon': 0.8895866370545631}. Best is trial#14 with value: 5.769943384670164.
[I 2020-05-06 08:35:36,176] Finished trial#36 with value: 5.777268521904065 with parameters: {'gamma': 5.913734617223292, 'c': 22830.465496459623, 'epsilon': 0.8523938870867831}. Best is trial#14 with value: 5.769943384670164.
[I 2020-05-06 08:35:36,262] Finished trial#37 with value: 5.775502618729382 with parameters: {'gamma': 6.183945838782351, 'c': 23183.369524071106, 'epsilon': 0.8013552931926867}. Best is trial#14 with value: 5.769943384670164.
[I 2020-05-06 08:35:36,351] Finished trial#38 with value: 5.773343092791149 with parameters: {'gamma': 6.785345050011865, 'c': 20728.975364505313, 'epsilon': 0.7373994796192317}. Best is trial#14 with value: 5.769943384670164.
[I 2020-05-06 08:35:36,443] Finished trial#39 with value: 5.774001041299327 with parameters:

[I 2020-05-06 08:35:39,319] Finished trial#70 with value: 5.781258262394715 with parameters: {'gamma': 2.8029054519011978, 'c': 2865.9193284744147, 'epsilon': 0.9728443669569963}. Best is trial#14 with value: 5.769943384670164.
[I 2020-05-06 08:35:39,422] Finished trial#71 with value: 5.782451821654198 with parameters: {'gamma': 3.5776626391239974, 'c': 66.69308018851396, 'epsilon': 0.9951354130181902}. Best is trial#14 with value: 5.769943384670164.
[I 2020-05-06 08:35:39,518] Finished trial#72 with value: 5.779840568111963 with parameters: {'gamma': 4.0924009697329, 'c': 1315.955923978913, 'epsilon': 0.9232131608989314}. Best is trial#14 with value: 5.769943384670164.
[I 2020-05-06 08:35:39,614] Finished trial#73 with value: 5.779559485643877 with parameters: {'gamma': 3.65600372553819, 'c': 905.75402103166, 'epsilon': 0.9192279683647453}. Best is trial#14 with value: 5.769943384670164.
[I 2020-05-06 08:35:39,711] Finished trial#74 with value: 5.778678271720536 with parameters: {'gam

[I 2020-05-06 08:35:42,637] Finished trial#105 with value: 5.758380079177787 with parameters: {'gamma': 4.74641986052691, 'c': 10049.5520396438, 'epsilon': 0.31335632911558337}. Best is trial#103 with value: 5.754830270753337.
[I 2020-05-06 08:35:42,734] Finished trial#106 with value: 5.758820049878043 with parameters: {'gamma': 5.095098015496481, 'c': 9973.661913167653, 'epsilon': 0.32729041540304193}. Best is trial#103 with value: 5.754830270753337.
[I 2020-05-06 08:35:42,832] Finished trial#107 with value: 5.7610867099354035 with parameters: {'gamma': 5.126472045660377, 'c': 12216.212028337115, 'epsilon': 0.3976739831727045}. Best is trial#103 with value: 5.754830270753337.
[I 2020-05-06 08:35:42,931] Finished trial#108 with value: 5.756680481399195 with parameters: {'gamma': 5.473800455891384, 'c': 7591.739861759466, 'epsilon': 0.2574261670022108}. Best is trial#103 with value: 5.754830270753337.
[I 2020-05-06 08:35:43,033] Finished trial#109 with value: 5.758276364522386 with para

Logo, como resultado os melhores parâmetros encontrados durante a busca são mostrados:

In [17]:
study.best_params

{'c': 9680.379348397655,
 'epsilon': 0.053667070084940545,
 'gamma': 4.971428800894493}

Bem como o resultado para o  erro absoluto médio (MAE) no conjunto de teste com esses parâmetros:

In [18]:
study.best_value

5.750421973943699

## CMA-ES

Semelhantemente ao algoritmo de Simullate Annealing, a implementação do algoritmo de otimização CMA-ES também foi feita via biblioteca optunza. A implementação porém é encapsulada em um sampler default fornecido pela própria biblioteca, assim não sendo necessário a implementação de uma classe sampler específica, somente a função objetivo.

In [19]:
def objective(trial):
    c = trial.suggest_uniform('c', 2**(-5),  2**15)
    gamma = trial.suggest_uniform('gamma', 2**(-15),  2**3)    
    epsilon = trial.suggest_uniform('epsilon', 0.05,  1.0)
    
    svr  = SVR(kernel='rbf', C=c, gamma=gamma, epsilon=epsilon)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae


sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=125)


[I 2020-05-06 08:35:44,797] Finished trial#0 with value: 5.781082688390094 with parameters: {'gamma': 2.7354294574213736, 'c': 1170.1785162100878, 'epsilon': 0.9689977970662065}. Best is trial#0 with value: 5.781082688390094.
[I 2020-05-06 08:35:44,873] Finished trial#1 with value: 5.765516977942874 with parameters: {'gamma': 3.941938467253303, 'c': 16383.908689983418, 'epsilon': 0.5346703869093998}. Best is trial#1 with value: 5.765516977942874.
[I 2020-05-06 08:35:44,952] Finished trial#2 with value: 5.76705177642033 with parameters: {'gamma': 4.0694676911697885, 'c': 16384.231438223913, 'epsilon': 0.5760157004334815}. Best is trial#1 with value: 5.765516977942874.
[I 2020-05-06 08:35:45,030] Finished trial#3 with value: 5.762718170828164 with parameters: {'gamma': 4.229428947097892, 'c': 16383.843332850987, 'epsilon': 0.4511939414862756}. Best is trial#3 with value: 5.762718170828164.
[I 2020-05-06 08:35:45,106] Finished trial#4 with value: 5.766713135828819 with parameters: {'gamma

[I 2020-05-06 08:35:47,609] Finished trial#35 with value: 5.751303376656554 with parameters: {'gamma': 4.106988159951556, 'c': 16383.856792167453, 'epsilon': 0.09292937406209462}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:47,694] Finished trial#36 with value: 5.757339619542418 with parameters: {'gamma': 3.873767037648648, 'c': 16383.915453199057, 'epsilon': 0.2895577479043259}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:47,779] Finished trial#37 with value: 5.7547056441346225 with parameters: {'gamma': 4.1901361286124015, 'c': 16383.800758881875, 'epsilon': 0.2025032534177917}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:47,866] Finished trial#38 with value: 5.754218801220518 with parameters: {'gamma': 4.072542162752177, 'c': 16383.506964036791, 'epsilon': 0.18815547481506417}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:47,973] Finished trial#39 with value: 5.755516108204418 with par

[I 2020-05-06 08:35:50,500] Finished trial#69 with value: 5.755019905678445 with parameters: {'gamma': 5.0386336320719645, 'c': 16383.595595252791, 'epsilon': 0.205697266028055}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:50,596] Finished trial#70 with value: 5.752830465267761 with parameters: {'gamma': 4.603243958704027, 'c': 16383.707914526685, 'epsilon': 0.13719452731451626}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:50,682] Finished trial#71 with value: 5.754471061214408 with parameters: {'gamma': 4.545891561506097, 'c': 16383.71258751951, 'epsilon': 0.19118866404993262}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:50,765] Finished trial#72 with value: 5.750703174156434 with parameters: {'gamma': 4.441397543390229, 'c': 16383.338595233989, 'epsilon': 0.06934445204729196}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:50,851] Finished trial#73 with value: 5.750298412838061 with param

[I 2020-05-06 08:35:53,751] Finished trial#103 with value: 5.751355333670778 with parameters: {'gamma': 4.78237321637897, 'c': 16383.080569767344, 'epsilon': 0.08745811183923662}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:53,842] Finished trial#104 with value: 5.758098227565568 with parameters: {'gamma': 6.1962066985563675, 'c': 16382.525864570409, 'epsilon': 0.296498556666144}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:53,943] Finished trial#105 with value: 5.755313158160351 with parameters: {'gamma': 4.766447827420908, 'c': 16383.107480759549, 'epsilon': 0.21600889365094986}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:54,040] Finished trial#106 with value: 5.7596126203267906 with parameters: {'gamma': 7.888011418318197, 'c': 16382.694941946405, 'epsilon': 0.3315118725636468}. Best is trial#25 with value: 5.7502468282164525.
[I 2020-05-06 08:35:54,131] Finished trial#107 with value: 5.7729273242697605 with

Logo, como resultado os melhores parâmetros encontrados durante a busca são mostrados:

In [20]:
study.best_params

{'c': 16384.62037224312,
 'epsilon': 0.07913982450497392,
 'gamma': 2.0429834996875442}

Bem como o resultado para o  erro absoluto médio (MAE) no conjunto de teste com esses parâmetros:

In [21]:
study.best_value

5.749751889571951